# Introduction
GOAL: To train a model to predict the wild fire risk of properties, using census data as input features and the proximity to fires as a target feature.

It would be cool to add in data regarding local climate.

In [10]:
import census
import censusgeocode as cg
import geopandas as gpd
import os
import numpy as np
import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# from censusdis.states import ALL_STATES_AND_DC


import load_GIS 
import load_census
import load_properties


from censusgeocode import CensusGeocode
from random_address import real_random_address


from IPython.display import IFrame
# from pygris import tracts
from matplotlib.colors import to_hex
from scipy.stats import randint, uniform
from shapely.geometry import Point
from shapely.ops import nearest_points
from shapely import distance
# from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestRegressor 
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import make_scorer, mean_poisson_deviance, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from xgboost import XGBRegressor



CRS = 5070
# ALL_STATES_AND_DC

# Load Training and Target Features

## Select Properties of Interest

Chosing 300000 properties randomly from US addresses. We will join relevant census data to these addresses. This will probably take awhile, so best to run it overnight.

We don't care about the address itself. We add a census identifier called the GEOID which based on the coordinate's state, county, and tract number.

Using a package that makes use of the [US Census Geocoder API](https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/census-geocoder.html), requests can be in batches of 10,000.

https://pypi.org/project/random-address/

TODO: How to speed this up? This feels like a very naive approach, atm. Takes about 2s per address. Yuck.

In [ ]:

num_properties = 1000  # 3000000
filepath = os.path.join('Data', 'property_coords.shp')
start = time.time()

# if not os.path.exists(filepath):
temp_lst = []
for _ in range(num_properties):

    coords = real_random_address()['coordinates']
    
    lat = coords['lat']
    long=coords['lng']
    resp = cg.coordinates(x=long, y=lat)
    # print(resp['Counties'])#.keys())
    # raise
    geoid = resp['2020 Census Blocks'][0]['GEOID']
    county= resp['Counties'][0]['BASENAME']
    state_code = resp['States'][0]['STATE']
    state_abb = resp['States'][0]['STUSAB']
    
    temp_lst.append(
        {
        'geometry' :   Point(long, lat),
        'geoid' :geoid,
        'county':county,
        'state_code':state_code,
        'state_abb':state_abb
        }
       )
properties_gpd = gpd.GeoDataFrame(temp_lst, crs=CRS)
properties_gpd.to_file(filepath,mode='a')
# else:
properties_gpd= gpd.read_file(filepath).to_crs(CRS)
end = time.time()
print(end - start)
properties_gpd.head()


Building the properties list over time, multiple runs, should drop dupes.


## Load 2023 US Census Data

Using an API key, we will use the 'census' Python package to interact with the US Govermnent's census API.

In [2]:
tracts=properties_gpd['tracts'].drop_duplicates()
print(tracts)



census = load_census.CensusData(tracts)


TypeError: LoadCensus.__init__() missing 1 required positional argument: 'filepaths'

## Load Wildfire GIS Data for 2024

We will use point data from the Visible Infrared Imaging Radiometer Suite (VIIRS). A valid alternative is using burn boundary data. There are a few different data sources we could use, but in the interest of (portfolio) simplicity we'll use just the VIIRS.

N:B: May be a good chance to practice using AWS DB storage and retrieval?

In [ ]:
gis_filepaths=[]
wf_gis = load_GIS.GISData(gis_filepaths)

## Give Each Property a Wildfire Risk Score

Will be based on the proximity to wildfire points, weighted by the number of nearby fires, with a cutoff of 50km.

# Machine Learning Considerations
#### Scoring Methods

For the float risk score, we can use Mean Squared Error (MSE) or Root Mean Squared Error (RMSE). Since it's quadratic in difference between observations and predictions deviations, MSE strongly penalizes large misses, which would be expensive for the insurance company.

For the risk category counts, they appear to be Poisson distributed, so a Poisson loss-function is appropriate.

For any classification model with the binned risk categories, we want to make large misses costly (i.e. predicting a 1 when the category is a 10), since these would also be very costly to the insurance company. To be honest, MSE will work here as well, since the categories are just 

# Model Machine Learning


NB: A good chance to make use of AWS compute.


### Split Data into Features/Targets



In [ ]:
drop_col =[
    'within_MTBS', 'within_MADIS', 'within_WFIGS',
   'within_ba', 'within_ba_no_WFIGS', 'noaa20_score',
   'noaa20_count_within_radius', 'noaa20_avg_dist_km', 
   'snpp_score', 'risk_score_prediction', 'snpp_count_within_radius',
   'id', 'longitude','latitude', 'geometry', 'geo_id', 
   'STUSPS', 'NAMELSADCO', 'score_cat', 'snpp_avg_dist_km', 'geo_id','geometry'
  ]
ml_df = model_joined.copy().drop(columns=drop_col)
del(model_joined)
ml_df._consolidate_inplace()


In [ ]:
# Target columns
target_col = ml_df['sat_avg']
feature_cols = ml_df.drop(columns=['sat_avg'])

random_state = 77
X_train, X_test, y_train, y_test = train_test_split(feature_cols, target_col, test_size=0.2, random_state=random_state)


#### RandomForestRegressor


In [ ]:
# rfr_rand_fp =os.path.join("Models","model_pred_best_RFR_randomCV.sav")

# if not os.path.exists(rfr_rand_fp):
#     param_dist = {
#         "n_estimators":    randint(100, 1000),   
#         "max_depth":       randint(5, 50),       
#         "min_samples_split": randint(2, 11),    
#         "min_samples_leaf":  randint(1, 5), 
#         "max_features":    [ "sqrt", "log2"] 
#     }
    
#     rfr = RandomForestRegressor(random_state=random_state, n_jobs=-1)
    
#     random_srch = RandomizedSearchCV(
#         estimator=rfr,
#         param_distributions=param_dist,
#         n_iter=5,  # start with 20 to get a feel for time
#         scoring='neg_mean_squared_error',
#         cv=5, 
#         random_state=random_state,
#         n_jobs=-1,
#         verbose=5  # 1
#     )

#     random_srch.fit(X_train, y_train)
    
#     print('best rfr params:', random_srch.best_params_)
#     # print('best score:', -random_srch.best_score_)
#     best_rfr = random_srch.best_estimator_
    

#     pickle.dump(best_rfr, open(rfr_rand_fp, 'wb'))
# best_rfr = pickle.load(open(rfr_rand_fp,'rb'))

# y_pred = best_rfr.predict(X_train)
# print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_pred)))

# y_pred = best_rfr.predict(X_test)
# print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


#### XGBoost


In [ ]:
cv_choice = 3
n_job_choice = 20

xgb_rand_fp =os.path.join("Models","model_pred_bbest_XGB_randomCV_{}cv_{}job.sav".format(cv_choice, n_job_choice))

if not os.path.exists(xgb_rand_fp):
    
    # XGBRegressor
    param_dist = {
        'n_estimators':randint(100, 1000),
        'learning_rate':uniform(0.01, 0.29),
        'max_depth':randint(3, 12),
        'min_child_weight':randint(1, 10),
        'subsample':uniform(0.5, 0.5),
        'colsample_bytree':uniform(0.5, 0.5),
        'gamma':uniform(0, 0.5),
        'reg_alpha': uniform(0, 1),
        'reg_lambda':uniform(0, 1),
    }
    
    xgb = XGBRegressor(random_state=random_state, n_jobs=-1)
    
    random_srch = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=20,  # start with 20 to get a feel for time
        scoring='neg_mean_squared_error',
        cv=3, #5, 
        random_state=random_state,
        n_jobs=-1,
        verbose=5  # 1
    )
    
    random_srch.fit(X_train, y_train)
    print('best xgb params:', random_srch.best_params_)
    # print('best score:', -random_srch.best_score_)
    best_xgb = random_srch.best_estimator_
    pickle.dump(best_xgb, open(xgb_rand_fp, 'wb'))
    
best_xgb = pickle.load(open(xgb_rand_fp,'rb'))
y_pred = best_xgb.predict(X_train)
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_pred)))
y_pred = best_xgb.predict(X_test)
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

#### Extract Feature Weights


In [ ]:

booster = best_xgb.get_booster()

importance_dict = booster.get_score(importance_type='gain')

imp_series = (
    pd.Series(importance_dict)
      .sort_values(ascending=False)
)

imp_arr = pd.Series(best_xgb.feature_importances_, index=X_train.columns)
top10 = imp_arr.sort_values(ascending=False).head(10)
print(top10)

# Conclusion